In [ ]:
# Dans le terminal :
# wget https://storage.googleapis.com/chrome-for-testing-public/131.0.6778.87/linux64/chromedriver-linux64.zip
# unzip chromedriver-linux64.zip
# ls -l
# cd chromedriver-linux64
# sudo mv chromedriver /usr/local/bin/
# sudo chmod +x /usr/local/bin/chromedriver

!pip install selenium==4.6.0 # cette version permet de télécharger directement le bon driver
!pip install webdriver-manager==3.8.1

!sudo apt update
!sudo apt install -y libnss3 libx11-xcb1 libxcomposite1 libxcursor1 libxdamage1 libxi6 libxtst6 libxrandr2 libasound2 libatk1.0-0 libpangocairo-1.0-0 libgtk-3-0

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.service import Service

chrome_options = Options()
chrome_options.add_argument("--headless")  # sans interface graphique
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service('/usr/local/bin/chromedriver')
browser = webdriver.Chrome(service=service, options=chrome_options)
print("Chromedriver lancé avec succès !")

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]        
Hit:3 https://ppa.launchpadcontent.net/git-core/ppa/ubuntu jammy InRelease 
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease                    
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,222 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,331 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,458 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,225 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,741 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,514 kB]
Get:13 https://apt.postg

In [2]:
import time
browser.get("https://www.marmiton.org/")
time.sleep(5)

try:
    cookies = browser.find_element(By.ID, "didomi-notice-agree-button")
    cookies.click()
    time.sleep(2)
    print("cookies acceptés")
except NoSuchElementException:
    pass
# Si le pop-up des cookies apparaît on les accepte (sinon il faudrait s'abonner), sinon on ne fait rien
# Le temps de 5 secondes donne le temps aux cookies d'apparaître


In [3]:
# Ingrédients : tomate et épinards

recherche = browser.find_element(By.ID, "mrtn-header-search-input")
recherche.send_keys("tomate+epinards")
recherche.send_keys(Keys.RETURN)
time.sleep(2)

In [5]:
# Résultats :

recettes = browser.find_elements(By.CSS_SELECTOR, ".recipe-card__title")  # Sélecteur pour les titres des recettes

# Nombre de résultats souhaités 
n = 3

# Liste des titres des premières recettes
premieres_recettes = [recette.text for recette in recettes[:n]]

for i, recette in enumerate(premieres_recettes, start=1):
    print(f"{i}. {recette}")


#####
liens = [lien.get_attribute("href") for lien in browser.find_elements(By.CSS_SELECTOR, ".recipe-card-link")]

# Liste des premiers liens correspondnat aux recettes proposées
premiers_liens = [lien for lien in liens[:n]]

for i, lien in enumerate(premiers_liens, start=1):
    print(f"{i}. {lien}")

1. Muffin quinoa épinard tomates séchées
2. Tarte thon, tomates et épinards
3. crumble épinards tomates
1. https://www.marmiton.org/recettes/recette_muffin-quinoa-epinard-tomates-sechees_346962.aspx
2. https://www.marmiton.org/recettes/recette_tarte-thon-tomates-et-epinards_37322.aspx
3. https://www.marmiton.org/recettes/recette_crumble-epinards-tomates_72661.aspx


In [ ]:
# Dictionnaire: ingredients/quantites par recette
dico_recettes={}
for i in range (len(premiers_liens)):
    lien=premiers_liens[i] #on prend le i ème lien de la liste de liens
    browser.get(lien)
    time.sleep(3)
    note=browser.find_element(By.CSS_SELECTOR, "span.recipe-header__rating-text")
    note=note.text
    nb_personnes=browser.find_element(By.CSS_SELECTOR, "input.recipe-ingredients__qt-counter__value.title-5")
    nb_personnes = nb_personnes.get_attribute("value")
    temps=browser.find_element(By.CSS_SELECTOR, "div.recipe-primary__item>span")
    temps=temps.text
    liste_ingredients={}
    ingredients=browser.find_elements(By.CSS_SELECTOR, "div.card-ingredient")
    #print(ingredients)
    for ingredient in ingredients:
        ingr_nom=ingredient.get_attribute("data-name")
        #ingr_nom=ingredient
        #print("ingr_nom", ingr_nom)
        try :
            quantite=ingredient.find_element(By.CSS_SELECTOR, "span.count").text
            #print("quantité", quantite)
            
        except:
            quantite=None #si la quantité n'est pas indiquée, par exemple pour le sel ou le poivre

        try:
            unite=ingredient.find_element(By.CSS_SELECTOR, "span.unit").text
            #print("unité", unite)
            
        except:
            unite=None #s'il n'y a pas d'unité, pour les fruits ou légumes par exemple


        elements={"quantité":quantite, "unité":unite}
        liste_ingredients[ingr_nom]=elements

        dico_recettes[premieres_recettes[i]]=liste_ingredients

    #for ingredient in ingredients:
     #   print(f"Ingrédient: {ingredient['ingredient']}, Quantité: {ingredient['quantite']}")
print(dico_recettes)

{'Muffin quinoa épinard tomates séchées': {'sel': {'quantité': '', 'unité': ''}, 'poivre': {'quantité': '', 'unité': ''}, 'cumin en poudre': {'quantité': '0.5', 'unité': 'cuillère à café'}, "piment d'Espelette": {'quantité': '1', 'unité': 'cuillère à café'}, 'persil': {'quantité': '', 'unité': ''}, 'oignon': {'quantité': '1', 'unité': ''}, 'tomate séchée': {'quantité': '100', 'unité': 'g'}, 'oeuf': {'quantité': '2', 'unité': ''}, 'quinoa': {'quantité': '100', 'unité': 'g'}, 'épinards': {'quantité': '200', 'unité': 'g'}, 'Parmesan': {'quantité': '50', 'unité': 'g'}, 'ail': {'quantité': '1', 'unité': 'gousse'}, "huile d'olive": {'quantité': '1', 'unité': 'cuillère à soupe'}}, 'Tarte thon, tomates et épinards': {'gruyère râpé': {'quantité': '150', 'unité': 'g'}, 'tomate': {'quantité': '2', 'unité': ''}, 'crème fraîche': {'quantité': '25', 'unité': 'cl'}, 'oignon': {'quantité': '2', 'unité': ''}, 'sel': {'quantité': '', 'unité': ''}, 'poivre': {'quantité': '', 'unité': ''}, 'beurre': {'qua

In [7]:
for recette, ingredients in dico_recettes.items():
    print(f"{recette}:")
    for ingredients, infos in ingredients.items():
        print(f"{ingredients}:")
        for cle, valeur in infos.items():
            print (f"{cle}:{valeur}")
        print()
    print()
    print()

Muffin quinoa épinard tomates séchées:
sel:
quantité:
unité:

poivre:
quantité:
unité:

cumin en poudre:
quantité:0.5
unité:cuillère à café

piment d'Espelette:
quantité:1
unité:cuillère à café

persil:
quantité:
unité:

oignon:
quantité:1
unité:

tomate séchée:
quantité:100
unité:g

oeuf:
quantité:2
unité:

quinoa:
quantité:100
unité:g

épinards:
quantité:200
unité:g

Parmesan:
quantité:50
unité:g

ail:
quantité:1
unité:gousse

huile d'olive:
quantité:1
unité:cuillère à soupe



Tarte thon, tomates et épinards:
gruyère râpé:
quantité:150
unité:g

tomate:
quantité:2
unité:

crème fraîche:
quantité:25
unité:cl

oignon:
quantité:2
unité:

sel:
quantité:
unité:

poivre:
quantité:
unité:

beurre:
quantité:150
unité:g

farine:
quantité:250
unité:g

oeuf:
quantité:4
unité:

eau:
quantité:2
unité:cuillères à soupe

épinards:
quantité:500
unité:g

lait:
quantité:10
unité:cl

thon:
quantité:1
unité:boîte



crumble épinards tomates:
sel:
quantité:
unité:

poivre:
quantité:
unité:

farine:
quant

In [31]:
import pandas as pd
import lecture as l
df_calories, df_svm = l.lecture_excel()
#df = pd.read_excel('https://ciqual.anses.fr/cms/sites/default/files/inline-files/Table%20Ciqual%202020_FR_2020%2007%2007.xls')
#df = df.loc[:,["alim_nom_fr","Energie, Règlement UE N° 1169/2011 (kcal/100 g)"]]
#df=df.rename({"Energie, Règlement UE N° 1169/2011 (kcal/100 g)":"calories_kcal_100g", "alim_nom_fr":"ingredients"}, axis=1) 
#df.iloc[:,1] = pd.to_numeric(df.iloc[:,1].str.replace(',', '.', regex=False))
print(df_calories.head())
def calcul_calories(ingredient, poids):
    resultat=df_svm[df_svm["Nom"]==ingredient]
    calories=resultat["Calories"]
    print(calories)

#print(calcul_calories("Oignon jaune, cru", 30))

                                          Groupe Sous-groupe Sous-sous-groupe  \
0                                            NaN         NaN              NaN   
338  Fruits, Légumes, Légumineuses Et Oléagineux     Légumes     légumes crus   
339  Fruits, Légumes, Légumineuses Et Oléagineux     Légumes     légumes crus   
340  Fruits, Légumes, Légumineuses Et Oléagineux     Légumes     légumes crus   
341  Fruits, Légumes, Légumineuses Et Oléagineux     Légumes     légumes crus   

                            Nom Calories Sucres Fibres Protéines Glucides  \
0       Dessert (aliment moyen)      NaN   23,7   1,54      4,61     36,6   
338          Avocat, pulpe, cru      205    0,4    3,6      1,56     0,83   
339       Bette ou blette, crue     16,4    1,2    1,8         1     1,63   
340               Carotte, crue     40,2      6    2,7      0,63     7,59   
341  Champignon, tout type, cru     21,7   1,43   1,72      2,37     1,88   

    Lipides  
0      12,9  
338    20,6  
339   < 